In [ ]:
import pickle

import pandas
import numpy as np
import matplotlib.pyplot as plt
import pandas.tools.plotting as pdplot

import stengel.model.pitch_data

In [ ]:
with open("../data/python/pitch_data.p", "rb") as f:
    pitcher_data = pickle.load(f)

### Distribution of Pitch Counts

Let's take a look at the distribution of pitch counts by pitcher. What do typical pitch counts over the course of a season look like?

In [ ]:
# Count pitches thrown by each pitcher
pitches_per_pitcher = [p.shape[0] for p in pitcher_data.values()]
pitches_per_pitcher.sort()

# Construct a histogram of pitch counts
plt.hist(pitches_per_pitcher, 50)
plt.show()

# Construct a quasi-CDF -- what percentage of pitches are thrown by pitchers with
# at least X pitches per season?
pitches_per_pitcher = np.array([pitches_per_pitcher])
pitches_cdf = np.cumsum(pitches_per_pitcher, 1, np.float32) / np.sum(pitches_per_pitcher)
pitches_cdf = np.concatenate((pitches_per_pitcher, pitches_cdf))
plt.plot(pitches_cdf[0, :], pitches_cdf[1, :])
plt.show()

So there's a chunk of elite starters with 2500-3500 pitches per season, a chunk of relievers/closers with 800-1300 pitches per season, plus an exponential smattering of pitchers with a handful of outings. Over 99% of pitches are thrown by pitchers with at least 100 pitches in a season.

### Pitchf/x Variable EDA

Let's do some EDA on the Pitchf/x variables. There are 22 variables in the Pitchf/x data, which is a lot to contend with. Our first task is to find some subset of the Pitchf/x variables that does a reasonable job of describing the range of a pitcher's repertoire of pitch types.

We can exclude `strike_zone_top` and `strike_zone_bottom` off the bat, since they're determined by the batter.  Likewise, we'll consider `plate_x` and `plate_z` (the position of the ball as it crosses the plane of the plate) to be orthogonal to pitch type. Finally, we can also exclude `start_y`, since according to Pitchf/x documentation, it's a fixed parameter of the system that the pitcher can't affect.

This is all of the low-hanging fruit that we can exclude *a priori*, but there are several other variables that are very highly correlated. Let's take a typical pitcher, and use his data for EDA:

In [ ]:
def pitcher_dataframe(pitcher):
    pitcher_cols = stengel.model.pitch_data.PitchDataGenerator.variable_names
    pitches = pitcher_data[pitcher]
    # Remove missing rows
    row_mins = np.amin(pitches, 1)
    complete_rows = np.array([i for i, x in enumerate(row_mins) if x is not None])
    df = list([list(x) for x in pitches])
    return pandas.DataFrame(df, columns=pitcher_cols)

In [ ]:
pitches = pitcher_dataframe("torrc001")

In [ ]:
pdplot.scatter_matrix(pitches.loc[:, ["start_speed", "end_speed", "velocity_y"]],
                      alpha=0.2, figsize=(4, 4), diagonal='kde')
plt.show()

Unsurprisingly, `start_speed`, `end_speed`, and `velocity_y` are all essentially collinear. `start_start` and `velocity_y` have the highest correlation, since they're measured at nearly the same point in time. We'll choose `velocity_y` for the sake of consistent units.

In [ ]:
pdplot.scatter_matrix(pitches.loc[:, ["break_y", "break_length", "break_angle", "delta_x", "delta_z", 
                                      "accel_x", "accel_z", "spin_direction", "spin_rate"]],
                      alpha=0.2, figsize=(8, 8), diagonal='kde')
plt.show()

`break_y` is very discrete, and therefore unlikely to be much help. Beyond that, it's clear that we have essentially four separate parameterizations of the movement of the ball (relative to normal expectations). `break_length` and `break_angle` can be thought of as the polar-coordinate representation of the movement, and `spin_direction` and `spin_rate` are polar-coordinate representations of the underlying process that causes the ball to accelerate. The other two parameterizations use Cartesian coordinates. The fact that `delta_*` and `accel_*` are so tightly coupled is no surprise, when we consider that no external forces (other than the Magnus effect) are working on the ball as it flies, so of course any deviation from a normal trajectory would manifest itself as acceleration early on. We prefer the `accel_*` parameterization for consistency's sake.

In [ ]:
pdplot.scatter_matrix(pitches.loc[:, ["start_x", "start_z", "plate_x", "plate_z", "accel_x", "accel_z"]],
                      alpha=0.2, figsize=(6, 6), diagonal='kde')
plt.show()

There's a bit of correlation between `start_x` and `start_z`, and `plate_x` and `plate_z`. The reason the correlation isn't stronger is likely because of the noise introduced by the other variables that influence pitch type -- for a consistent type of pitch thrown, the correlation between `start_*` and `plate_*` should be much higher. For the same reason that we prefer to exclude `plate_*`, we'll exclude `start_*` for the sake of the pitch classification model. Furthermore, it looks like `start_*` and `accel_*` are fairly orthogonal -- it looks like we aren't gaining extra insight into the pitcher's mechanics (and therefore his choice of pitch) from the coordinates of his release point.

This leaves us with a promising set of variables for pitch classification. Essentially, we could use the first and second derivatives of the ball's position in three dimensions at a particular point in its flight path to determine what kind of pitch it is. This is enormously satisfying, both statistically (using only six variables is immensely preferable to using 22), and mechanically (given that we consider a pitch's location to be orthogonal to how it was thrown, a complete description of its movement at a single point in time should be sufficient to reconstruct any other metric describing its flight path, at least to a first-order approximation).

In [ ]:
pdplot.scatter_matrix(pitches.loc[:, ["velocity_x", "velocity_y", "velocity_z", 
                                      "accel_x", "accel_y", "accel_z"]],
                      alpha=0.2, figsize=(6, 6), diagonal='kde')
plt.show()

However, we could reduce the number of variables in the model even further. `accel_y` is strongly related to `velocity_y` for fairly obvious reasons -- the faster a ball is moving, the greater the air resistance it will encounter, and the faster it will decelerate. Furthermore, there doesn't appear to be a lot of clustering segmentation due to `velocity_x` and `velocity_z`. Therefore, if we drop those three variables, we are left with three simple variables that do a very good job of clustering pitch types: `velocity_y` (how fast the pitch was thrown, and `accel_x` and `accel_z` (the pitch's movement relative to the plane of the strike zone, expressed in Cartesian coordinates).

In [ ]:
pdplot.scatter_matrix(pitches.loc[:, ["velocity_y", "accel_x", "accel_z"]],
                      alpha=0.2, figsize=(6, 6), diagonal='kde')
plt.show()

Note the natural appearance of fairly well-defined clusters within this scatterplot matrix. This makes sense, given our simple conceptual model of how the data is generated: a pitcher chooses of a few discrete pitch types in his repertoire, then executes it with some Gaussian-ish process variance.